[See docs/DISCLAIMER_SNIPPET.md](../../../docs/DISCLAIMER_SNIPPET.md)

# ðŸ›’ Alphaâ€‘AGI Marketplace Â· Colab Notebook

Run the Alphaâ€‘Factory marketplace micro-demo.

* ðŸ”§ Start the orchestrator
* ðŸ“¨ Queue a sample job and inspect memory
* ðŸ”´ Works offline via Mixtral unless you provide an `OPENAI_API_KEY`
* âœ… After setup run `python check_env.py --auto-install`

Paste your OpenAI key below or leave blank to run offline.

**By running this notebook you agree to the [demo terms](TERMS_AND_CONDITIONS.md).**

In [ ]:
OPENAI_API_KEY = ""  # optional

In [ ]:
%%bashif [ ! -d AGI-Alpha-Agent-v0 ]; then  git clone --depth 1 https://github.com/MontrealAI/AGI-Alpha-Agent-v0.gitficd AGI-Alpha-Agent-v0pip install -q -r alpha_factory_v1/requirements-colab.txt

### Offline wheelhouse install
Mount a Google Drive folder (or local path) containing pre-built wheels and install packages without contacting PyPI. The package list is locked in `alpha_factory_v1/requirements-colab.lock`.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
wheelhouse = '/content/drive/MyDrive/wheelhouse'  # or any local path
!pip install --no-index --find-links $wheelhouse -r AGI-Alpha-Agent-v0/alpha_factory_v1/requirements-colab.lock


## ðŸš€ Launch orchestrator (background)

In [ ]:
%cd AGI-Alpha-Agent-v0
!python check_env.py --auto-install


In [ ]:
%%bash --bg
cd AGI-Alpha-Agent-v0/alpha_factory_v1
uvicorn backend.orchestrator:app --host 0.0.0.0 --port 8000 &>/dev/null &

## ðŸ”— Expose API with pyngrok

In [ ]:
from IPython.display import Markdown, display
from pyngrok import ngrok, conf
conf.get_default().region = 'us'
api = ngrok.connect(8000, 'http')
display(Markdown(f'[Open API docs]({api.public_url}/docs)'))
print('API â†’', api.public_url)

## âœ… Check orchestrator status

In [ ]:
from alpha_factory_v1.demos.alpha_agi_marketplace_v1 import MarketplaceClient
client = MarketplaceClient()
print('health â†’', client.health())
print('agents â†’', client.agents())

## ðŸ“‹ Queue sample job and view memory

In [ ]:
from alpha_factory_v1.demos.alpha_agi_marketplace_v1 import MarketplaceClient, SAMPLE_JOB
import time, json
client = MarketplaceClient()
for _ in range(20):
    try:
        if client.health() == 'ok':
            break
    except Exception:
        time.sleep(1)

job = json.load(open(SAMPLE_JOB))
client.queue_job(job)
time.sleep(2)
print('recent memory â†’', client.recent_memory(job['agent']))

---
### ðŸŽ‰ All set!
Use the API URL above to monitor progress or trigger your own jobs.